# K2496 example

Quantify K2496, a Ba, Ti, Si and O containing engineered glass.

In [ ]:
# Load DrWatson project manager and enable custom environment
using DrWatson
@quickactivate "HyperspectraWithNeXL"
# Load 3rd party libraries for plotting and tabulation
using Gadfly, DataFrames
# Load the necessary NeXL libraries
using Revise
using NeXLSpectrum


path = joinpath(datadir(),"exp_raw","K2496")
# Read the spectra from disk
k2496 = loadspectrum.(joinpath(path,"K2496_$(i).msa") for i in 1:3)
benitoite = loadspectrum(joinpath(path,"Benitoite std.msa"))
# Plot them using a method of Gadfly.plot(...) specialized for Spectrum items
set_default_plot_size(8inch,3inch)
plot(k2496..., benitoite, klms=[ n"O", n"Si", n"Ti", n"Ba" ], xmax=6.0e3)

In [ ]:
properties(k2496[1])

Define a BasicEDS detector object with the properties of the measurement device.  Then apply the detector to the measured spectra.

In [ ]:
det = BasicEDS(4096, -480.40409, 5.00525, 132.0, 110, 
         Dict(KShell=>n"B", LShell=>n"Ca", MShell=>n"Cs"))
k2496 = map(s->apply(s,det), k2496)
benitoite = apply(benitoite,det) 
det

Compute the filtered reference spectra from the fitting standards.  In this case, we are using "Sanbornite std.msa" for the B L-family, "BaCl2 std.msa" for the Ba M-lines, "MgO std.msa" for O and pure elements for Si and Ti. 

In [ ]:
refs = references( [
  reference(n"Si", joinpath(path,"Si std.msa"), mat"Si"),
  reference(n"Ba", joinpath(path,"Sanbornite std.msa"), mat"BaSi2O5"),  # Only L-lines are clear
  reference(n"Ba", joinpath(path,"BaCl2 std.msa"), mat"BaCl2"), # Picks up the M-lines
  reference(n"Ti", joinpath(path,"Ti std.msa"), mat"Ti"),
  reference(n"O", joinpath(path,"MgO std.msa"), mat"MgO"),
  reference(n"C", joinpath(path,"C std.msa"), mat"C") ], det)
ENV["columns"]=400
asa(DataFrame, refs)

Fit (fit_spectra(...)) and then matrix correct (quantify(...)) the unknown spectra.

Fit these references to the K2496 to get a set of k-ratios with respect to the fitting standards.

In [ ]:
fs=fit_spectrum(k2496, refs)
asa(DataFrame, fs, withUnc=true)

Now quantify K2496 using the fitting standards and compare to the nominal composition.

In [ ]:
q=quantify.(fs, strip=[n"C"])
k2496_nom = parse(Material,"0.323*O+0.2291*Si+0.018*Ti+0.4299*Ba",name="K2496 nominal")
asa(DataFrame, q, nominal=k2496_nom)

Now we turn our attention to the quantification standard "Benitoite."  We will fit the same fitting standards to Benitoite and get a set of k-ratios for Benitoite relative to the fitting standards.

In [ ]:
benitoite_std = fit_spectrum(benitoite, refs)
asa(DataFrame, [ benitoite_std ], withUnc=true)

Next, we take into account the carbon coating on the Benitoite standard.  We will use the C K-L2 k-ratio to estimate the mass-thickness of the carbon coating.

In [ ]:
std_mat =  parse(Material, "BaTiSi3O9", name="Benitoite")
coating_mat = parse(Material,"C",density=1.9)
estimatecoating(benitoite_std, std_mat, coating_mat, n"C K-L2")

In [ ]:
plot(benitoite_std)

The propertes of `benitoite_std` show that the estimated mass-thickness of the carbon coating was assigned to the :Coating property for use later in quantification.

In [ ]:
properties(benitoite_std.label.spectrum)

In [ ]:
std_mat

Apply the quantification standard k-ratios in `benitoite_std` to the unknown materials k-ratios measured relative to the fitting standards `fsi` where the assumed composition of the standard is `std_mat`.

In [ ]:
fs_stds = map(fsi->standardize(fsi, benitoite_std, std_mat), fs)
asa(DataFrame, fs_stds, withUnc=true)

Now quantify the unknown k-ratio relative to the fitting standards.  In addition, use the measured k-ratio for C to estimate the coating thickness on the unknown.

In [ ]:
q_stds = quantify.(fs_stds, coating=n"C K-L2"=>coating_mat)
asa(DataFrame, q_stds, nominal=k2496_nom)

The plot may be saved in various formats (including SVG, PNG and PDF).

In [ ]:
using Cairo, Fontconfig
#plot(fs_stds[1]) |> SVG(joinpath(plotsdir(), "K2496 residual - indirect.svg"), 8inch, 3inch)
#plot(fs_stds[1]) |> PNG(joinpath(plotsdir(), "K2496 residual - indirect.png"), 8inch, 3inch)
plot(fs_stds[1]) |> PDF(joinpath(plotsdir(), "K2496 residual - indirect.pdf"), 8inch, 3inch)

Output the quant results.  Write the results to a CSV file.

In [ ]:
df = asa(DataFrame, q_stds, nominal=k2496_nom)
using CSV
CSV.write(joinpath(datadir(),"exp_pro","K2496 quant - Benitoite.csv"), df)
df

Since there were three measurements, we can use the `describe(...)` function to statistically summarize the measured compositions.

In [ ]:
using Statistics
zs=sort(collect(elms(k2496_nom)))
res_df=DataFrame(
    Elements=symbol.(zs), #
    Nominal=map(z->k2496_nom[z],zs), #
    Fitting=map(z->mean(qi->qi.comp[z], q), zs),
    Similar=map(z->mean(qi->qi.comp[z], q_stds), zs)
)
push!(res_df, ["Total", sum(res_df[:, :Nominal]), sum(res_df[:, :Fitting]), sum(res_df[:, :Similar])])

Tabulating a single IterationResult item provides extensive analysis details including uncertainties and matrix correction factors.

In [ ]:
asa(DataFrame, q_stds[1])

Compare this with the same details generated when the unknown was quantified using the fitting standards.  The matrix correction factors differ further from unity suggesting a less accurate measurement.

In [ ]:
asa(DataFrame, q[1])

Now let's generate LaTeX tables for the paper.

In [ ]:
using PrettyTables
tbl = vcat( (asa(DataFrame,qi) for qi in q_stds)...) 
sort!(tbl, :Element)
open(joinpath(papersdir(),"Tables","K2496benitoite.tex"),"w") do io
  pretty_table(io, tbl, nosubheader=true, backend=:latex, 
            label="tbl:k2496_benitoite")
end

open(joinpath(papersdir(),"Tables","K2496overview.tex"),"w") do io
    pretty_table(io, res_df, nosubheader=true, backend=:latex, label="tbl:k2496over")
end

QED